In [1]:
import pdfplumber
import os
import pandas as pd
from datetime import datetime
import json

with open('DICCIONARIO_CODIGO_NOMBRE_FARMACOS.json', 'r', encoding = 'utf-8') as f:
    DICCIONARIO_CODIGO_NOMBRE_FARMACOS = json.load(f)

In [15]:
class Formateador:
    def __init__(self):
        pass

    def hacer_tabla_global(self):
        self.antibiogramas = self.obtener_datos_todos_los_pacientes()
        columnas = ['Ingreso', 'Tipo muestra', 'Nº de Cultivo', 'Nº de Orden', 'Rut', 'Nombre', 'Servicio', 'Fecha Firma'] + list(DICCIONARIO_COLUMNAS_FARMACOS_EN_EXCEL.keys())

        for antibiograma in self.antibiogramas:
            lista_antibiograma = antibiograma.pop()
            for dato in lista_antibiograma:
                antibiograma.append(dato)

        df = pd.DataFrame(self.antibiogramas, columns = columnas)
        return df


    def obtener_datos_todos_los_pacientes(self):
        pacientes = []
        for nombre_archivo in os.listdir():
            if '.pdf' in nombre_archivo:
                paciente = self.obtener_datos_un_paciente(nombre_archivo)
                pacientes.append(paciente)
        

        antibiogramas = [paciente for paciente in pacientes if type(paciente[-1]) == list]
        return antibiogramas


    def obtener_datos_un_paciente(self, nombre_archivo_paciente):
        lista_datos_personales = self.obtener_datos_personales_paciente(nombre_archivo_paciente)
        lista_resultados_sensibilidad = self.obtener_tabla_antibiograma(nombre_archivo_paciente)

        return [*lista_datos_personales, lista_resultados_sensibilidad]

    # Esta función obteine los datos personales de un paciente, retorna una tupla.
    def obtener_datos_personales_paciente(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            datos_personales_relevantes = pdf.pages[0].extract_text().split('\n')[3:12]

            nombre_paciente = datos_personales_relevantes[0].split(':')[1][:-10]
            n_orden = datos_personales_relevantes[0].split(':')[-1]
            rut = datos_personales_relevantes[1].split(':')[-1]

            linea_ingreso = datos_personales_relevantes[4].split(' ')
            try:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_ingreso = datetime.strptime(f'{linea_ingreso[-2]} {linea_ingreso[-1]}', ':%d/%m/%Y %H:%M:%S')


            linea_firma = datos_personales_relevantes[5].split(' ')
            try:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d-%m-%Y %H:%M:%S')
            except ValueError:
                fecha_firma = datetime.strptime(f'{linea_firma[-2]} {linea_firma[-1]}', ':%d/%m/%Y %H:%M:%S')

            seccion = datos_personales_relevantes[6].split(':')[-1]
            tipo_muestra = datos_personales_relevantes[7].split(':')[-1]
            n_cultivo = datos_personales_relevantes[8].split(':', 1)[-1]

            return [fecha_ingreso, tipo_muestra, n_cultivo, n_orden, rut, nombre_paciente, seccion, fecha_firma]

    # Esta función itera en todo un pdf, para buscar las tablas que sean de antibiograma.
    def obtener_tabla_antibiograma(self, nombre_archivo_paciente):
        with pdfplumber.open(nombre_archivo_paciente) as pdf:
            for pagina in pdf.pages:
                tablas_totales_de_pagina = pagina.extract_table()
                if tablas_totales_de_pagina:
                    for tabla in tablas_totales_de_pagina:
                        tabla = tabla[0].split('\n')
                        if 'ANTIBIOGRAMA' in tabla:
                            lista_sensibilidades = self.formatear_tabla_antibiograma(tabla)
                            return lista_sensibilidades

                else:
                    return None 

    # Esta función construye la tabla de antibiograma para hacerla un Df. Retorna un Df
    def formatear_tabla_antibiograma(self, lista_tabla):
        separados = list(map(lambda x: x.split(' '), lista_tabla))
        nombre_tabla, headers = separados[0], separados[1]
        numero_cepas = [dato for dato in headers if dato.isnumeric()]

        columnas_headers = ['ANTIBIOTICOS']
        for numero_cepa in numero_cepas:
            columnas_headers.append(f'Cepa {numero_cepa}')
            columnas_headers.append(f'CIM')
        

        datos_susceptibilidad = separados[2:]
        for dato in datos_susceptibilidad:
            espacios_a_llenar_en_linea = len(columnas_headers) - len(dato)
            dato += [None] * espacios_a_llenar_en_linea
 
        for dato in datos_susceptibilidad:
            dato[0] = DICCIONARIO_CODIGO_NOMBRE_FARMACOS[dato[0]]


        diccionario_sensibilidades_a_llenar = {farmaco: None for farmaco in DICCIONARIO_CODIGO_NOMBRE_FARMACOS.values()}
        for farmaco in datos_susceptibilidad:
            diccionario_sensibilidades_a_llenar[farmaco[0]] = farmaco[1] 

        lista_sensibilidades_llenas = list(diccionario_sensibilidades_a_llenar.values())

        return lista_sensibilidades_llenas
    


In [16]:
formateador = Formateador()
df = formateador.hacer_tabla_global()

In [18]:
df.to_excel('Prueba.xlsx')

In [17]:
df

,Ingreso,Tipo muestra,Nº de Cultivo,Nº de Orden,Rut,Nombre,Servicio,Fecha Firma,AK,AMP,...,SAM,SP,TAZO,TEICO,TGC,TOB,LEVO,MINO,DAP,VAN
0,2022-04-28 12:50:05,Expectoración (M1),942658,00942658,22.189.976-8,MARTINA ANGELA DIAZ SASSO,Microbiología,2022-05-03 12:13:02,Resistente,None,...,None,None,Sensible,None,None,None,None,None,None,Sensible
1,2022-05-03 09:53:01,UROCULTIVO,943532,00943532,6.639.095-0,NORMA LÓPEZ LEYTON,Microbiología,2022-05-05 13:30:39,Sensible,None,...,None,None,Sensible,None,None,None,None,None,None,None
2,2022-05-03 11:38:23,Lavado Bronquial,943594,00943594,5.221.257-K,MARIA ARREPOL RIFFO,Microbiología,2022-05-05 13:38:22,None,None,...,None,None,None,None,None,None,None,None,None,Sensible
3,2022-05-03 11:39:22,Lavado Bronquial,943596,00943596,5.221.257-K,MARIA ARREPOL RIFFO,Microbiología,2022-05-05 13:36:17,None,None,...,None,None,None,None,None,None,None,None,None,Sensible
4,2022-05-03 11:58:34,Expectoración (M1),943606,00943606,18.353.213-8,JAIME BENJAMIN HEREDIA MAZU,Microbiología,2022-05-06 11:19:51,None,None,...,None,None,None,None,None,None,None,None,None,Sensible
5,2022-05-04 10:03:46,Expectoración (M1),943814,00943814,15.289.767-7,CATALINA ELENA RUEDA CORTEZ,Microbiología,2022-05-06 11:24:03,None,None,...,None,None,None,None,None,None,None,None,None,Sensible
6,2022-05-04 10:41:48,Expectoración (M1),943840,00943840,20.297.859-2,CRISTIAN IGNACIO RAMOS ENSIS,Microbiología,2022-05-06 11:31:12,None,None,...,None,None,None,None,None,None,None,None,None,Sensible
7,2022-05-04 11:02:27,Expectoración (M1),943862,00943862,6.000.629-6,MARIA ROSA MARTINEZ MESIAS,Microbiología,2022-05-06 11:39:05,None,None,...,None,None,None,None,None,None,None,None,None,None
8,2022-05-04 11:57:13,Lavado Bronquial,943891,00943891,3.926.981-3,OLGA DEL CARMEN VERDUGO CASTILLO,Microbiología,2022-05-06 11:41:51,None,None,...,Sensible,None,None,None,None,None,None,None,None,None
9,2022-05-04 14:54:54,Aspirado Endotraqueal,943953,00943953,10.495.611-4,GABRIEL DEL CARMEN GUZMAN DIAZ,Microbiología,2022-05-06 11:44:22,None,None,...,None,None,None,None,None,None,None,None,None,None
